# MNIST

In [17]:
import torch
import torchvision
import torch.utils.data
import torchvision.transforms as transforms

## Load data from torchvision 

In [18]:
# Transform the data to torch tensors and normalize it 
transform = transforms.Compose([transforms.ToTensor(),
transforms.Normalize((0.1307), ((0.3081)))])

# Prepare training set and testing set
trainset = torchvision.datasets.MNIST('mnist', train=True, 
download=True, transform=transform)
testset = torchvision.datasets.MNIST('mnist', train=False, 
download=True, transform=transform)

# Prepare training loader and testing loader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
shuffle=True, num_workers=0)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
shuffle=False, num_workers=0)

In [19]:
# check shape
print(trainloader.dataset.train_data.shape)
print(testloader.dataset.test_data.shape)

torch.Size([60000, 28, 28])
torch.Size([10000, 28, 28])


/Users/menghsuanlee/opt/anaconda3/lib/python3.9/site-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/Users/menghsuanlee/opt/anaconda3/lib/python3.9/site-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


In [11]:
# check batch size
print(testloader.batch_size)

32


## Train model 

Steps:
1. Build NN
2. Loop over:
    - foward pass
    - calculate loss
    - calculate gradients
    - change weights based on gradients( weights -= weight_gradient*learning_rate)
   

In [10]:
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

In [20]:
# Define the class Net, inherent from nn.Module
class Net(nn.Module):
    def __init__(self):    
        # Define all the parameters of the net
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28 * 1, 200) # input 28*28, 200 units in hidden layer
        self.fc2 = nn.Linear(200, 10) # 10 classes output

    def forward(self, x):    
        # Do the forward pass
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [22]:
# Instantiate the Adam optimizer and Cross-Entropy loss function
model = Net()   
optimizer = optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()
  
for batch_idx, data_target in enumerate(trainloader):
    data = data_target[0]
    target = data_target[1]
    data = data.view(-1, 28 * 28) # put all entries into vectors
    
    # zero the gradients, not to accumulate gradients from last iteration
    optimizer.zero_grad()

    # Complete a forward pass
    output = model(data)

    # Compute the loss, gradients and change the weights
    loss = criterion(output, target)
    
    # backward prop: compute the gradient
    loss.backward()
    
    # change the weights
    optimizer.step()

In [25]:
total = 0
correct = 0

# Set the model in eval mode
model.eval()

for i, data in enumerate(testloader, 0):
    inputs, labels = data
    
    # Put each image into a vector
    inputs = inputs.view(-1, 28 * 28)
    
    # Do the forward pass and get the predictions
    outputs = model(inputs)
    # take maximum score as the prediction
    _, outputs = torch.max(outputs.data, 1)
    
    # update counter
    total += labels.size(0)
    correct += (outputs == labels).sum().item()
print('The testing set accuracy of the network is: %d %%' % (100 * correct / total))

The testing set accuracy of the network is: 94 %
